In [ ]:
!pip install anvil-uplink


In [ ]:
!pip install openai

In [ ]:
!pip install plantuml

In [ ]:
import openai
from plantuml import PlantUML


# Функция для извлечения кода PlantUML диаграмм из ответа модели
def extract_plantuml_code(text):
    start_index = text.find("@startuml")
    end_index = text.find("@enduml")
    if start_index != -1 and end_index != -1:
        return text[start_index:end_index + len("@enduml")].strip()
    return None

#функция для генерации UMl изображений
def generate_plantuml_image(plantuml_code, output_file):
    # Создаем экземпляр PlantUML с URL сервера
    plantuml = PlantUML(url='http://www.plantuml.com/plantuml/png/')

    # Записываем код PlantUML во временный файл
    with open("temp.uml", "w") as temp_file:
        temp_file.write(plantuml_code)

    # Генерируем изображение и сохраняем его в указанный файл
    plantuml.processes_file("temp.uml", output_file)

# API ключ
openai.api_key = ""

# Функция для получения ответа от GPT
def get_assistant_response(messages):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": m["role"], "content": m["content"]} for m in messages]
    )
    return response.choices[0].message["content"]

def get_PlantUML_code(user_input):
    # Инициализация истории сообщений
    messages = [{"role": "assistant", "content": "Как я могу помочь?"}]

    # Запрашиваем генерацию кода PlantUML для диаграммы вариантов использования
    use_case_prompt = f"I want you to act as a professional system analyst and designer with specialized proficiency in generating PlantUML diagrams. \
    Based on the following system description, generate a PlantUML use case diagram code. Ensure that your diagram aligns with the system description and \
    adheres to standard PlantUML syntax and best practices. Don't include any explanations in your responses. Here is the system description: {user_input}"
    messages.append({"role": "user", "content": use_case_prompt})
    use_case_response = get_assistant_response(messages)
    messages.append({"role": "assistant", "content": use_case_response})

    # Запрашиваем генерацию кода PlantUML для диаграммы классов
    class_prompt = f"I want you to act as a professional system analyst and designer with specialized proficiency in generating PlantUML diagrams.\
     Based on the following system description, generate a PlantUML class diagram code. Ensure that your diagram aligns with the system description\
      and adheres to standard PlantUML syntax and best practices.  Don't include any explanations in your responses. Here is the system description: {user_input}"
    messages.append({"role": "user", "content": class_prompt})
    class_response = get_assistant_response(messages)
    messages.append({"role": "assistant", "content": class_response})

    # Запрашиваем генерацию кода PlantUML для диаграммы последовательности
    sequence_prompt = f"I want you to act as a professional system analyst and designer with specialized proficiency in generating PlantUML diagrams. \
    Based on the following system description, generate a PlantUML sequence diagram code. \
    Ensure that your diagram aligns with the system description and adheres to standard PlantUML syntax and best practices.  Don't include any explanations in your responses. \
    Here is the system description: {user_input}"
    messages.append({"role": "user", "content": sequence_prompt})
    sequence_response = get_assistant_response(messages)
    messages.append({"role": "assistant", "content": sequence_response})

    # Извлекаем и сохраняем код диаграмм
    plantuml_code = {
        "use_case": extract_plantuml_code(use_case_response),
        "class": extract_plantuml_code(class_response),
        "sequence": extract_plantuml_code(sequence_response)
    }
    return plantuml_code

